In [11]:
from pathlib import Path
from zero_data.io_list.readers.marpower import MarpowerReader
import polars as pl
df = MarpowerReader().read_io_list(Path("../io_lists/52422003_3210_AMCS IO-List R2.1.xlsx")).io_list
df

rev.,deleted,target_type,device,module,module_type,terminal,is_subscribe,prefix_device,tag,cabinet,opc_ua_node_id,opc_ua_publish_interval,system,yard_tag,description,parent,redundant_tag,p&id,cable_id,cable_type,cable_core_no.,mqtt_topic,mqtt_json_path,modbus_slave_address,modbus_data_type,modbus_address,modbus_source_type,modbus_byte_size,modbus_swap_words,modbus_lower,modbus_upper,range_lower,range_upper,unit,precision,alert_code,alert_hh,alert_h,alert_l,alert_ll,alert_f,delay_on,acknowledge_location,sounding_locations,alert_priority,intended_operator_response,category_a,group_alarm,call_gea_on_alert_,disallow_inhibit,vdr_id,general_lock_tag,general_lock_operator,general_lock_level,hh_lock_tag,hh_lock_operator,hh_lock_level,h_lock_tag,h_lock_operator,h_lock_level,l_lock_tag,l_lock_operator,l_lock_level,ll_lock_tag,ll_lock_operator,ll_lock_level,do_not_log,log_to_daily_report,log_to_cdp,workstation,timestamp,data_type
str,str,str,str,str,str,str,null,str,str,str,null,str,str,str,str,null,null,null,str,str,null,null,null,str,str,str,null,null,null,null,null,str,str,str,null,str,str,str,str,null,str,str,str,str,str,str,null,str,null,null,str,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,str
"""R1.14_FAT""",null,"""Bool""","""KEB1""","""R1""","""750-626/020-002""","""0""",null,"""True""","""GFDEnabled""","""DSK_SWR""",null,"""1000""","""4500 AMCS""",null,"""MTS AMCS KEB1 Ground Fault Dia…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""BOOLEAN"""
"""R1.14_FAT""",null,"""Bool""","""KEB1""","""R1""","""750-626/020-002""","""2""",null,"""True""","""FieldVoltageAvailable""","""DSK_SWR""",null,"""1000""","""4500 AMCS""",null,"""MTS AMCS KEB1 Voltage Availabl…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""BOOLEAN"""
"""R1.14_FAT""",null,"""Bool""","""KEB1""","""R1""","""750-626/020-002""","""4""",null,"""True""","""PA24V""","""DSK_SWR""",null,"""1000""","""4500 AMCS""",null,"""MTS AMCS KEB1 Power Distributi…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""6""",null,null,null,null,"""True""",null,"""TS""",null,"""Alarm""","""Check for earth fault""",null,null,null,null,"""1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""BOOLEAN"""
"""R1.14_FAT""",null,"""Bool""","""KEB1""","""R1""","""750-626/020-002""","""5""",null,"""True""","""MA24V""","""DSK_SWR""",null,"""1000""","""4500 AMCS""",null,"""MTS AMCS KEB1 Power Distributi…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""7""",null,null,null,null,"""True""",null,"""TS""",null,"""Alarm""","""Check for earth fault""",null,null,null,null,"""1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""BOOLEAN"""
"""R1.14_FAT""",null,"""Bool""","""KEB1""","""R1""","""750-626/020-002""","""6""",null,"""True""","""PA0V""","""DSK_SWR""",null,"""1000""","""4500 AMCS""",null,"""MTS AMCS KEB1 Power Distributi…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""8""",null,null,null,null,"""True""",null,"""TS""",null,"""Alarm""","""Check for earth fault""",null,null,null,null,"""1""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""BOOLEAN"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

The goal of this notebook is to explore the Eekels IO list and compare different ways to group the fields in to topics:
- Yard tag
- System
- Device+Module

## Yard tag
Yard tag is a unique identifier for a component.

In [12]:
df.group_by("yard_tag").len("number_of_fields").group_by("number_of_fields").len("number_of_topics")

number_of_fields,number_of_topics
u32,u32
5,2
4,3
1,94
2,16
299,1
3,8


In [13]:
df.filter(pl.col("yard_tag").is_null()).select(pl.len().alias("null_count"))

null_count
u32
299


## System
System is a functional tag that is given by Vitters. The advantage of using this to group our topics is that the fields are grouped based on function.

In [14]:
df.group_by("system").len("number_of_fields").group_by("number_of_fields").len("number_of_topics")

number_of_fields,number_of_topics
u32,u32
42,1
1,5
20,1
54,1
17,1
…,…
33,1
14,1
2,3


In [15]:
df.group_by("system").len("number_of_fields").sort("number_of_fields", descending=True)

system,number_of_fields
str,u32
"""4500 AMCS""",95
"""Navigation Lights""",80
"""2500 FRESH WATER""",54
"""3500 VENTILATION""",42
"""0900 DOORS HATCHES""",38
…,…
"""AMCS - CTRL""",1
"""2900 PNEUMATIC """,1
"""2102 BILGE FIFI""",1


## Device - Module
This is a combination of the AMCS device and the PLC module it is connected to and has no functional meaning.

In [16]:
df.group_by("device", "module").len("number_of_fields").group_by("number_of_fields").len("number_of_topics").sort("number_of_fields", descending=True)

number_of_fields,number_of_topics
u32,u32
80,1
16,2
8,25
7,2
6,6
4,22
3,1
2,8
1,2


In [17]:
df.group_by("device", "module").len("number_of_fields").sort("number_of_fields", descending=True)

device,module,number_of_fields
str,str,u32
"""NavLights""",null,80
"""KEB1""","""DO61""",16
"""KEB1""","""DO60""",16
"""KEB1""","""DO00""",8
"""KEB1""","""DI29""",8
…,…,…
"""KEB2""","""AO60""",2
"""KEB1""","""AI31""",2
"""KEB1""","""AO60""",2


# IO List proposition
We would prefer that the data is grouped by function in to topics. Having large topics includes more data duplication. So our preference would go to smaller topics. To make choices we have come up with a few principles:
- Group by "System" (2500 TECHWATER, 4500 AMCS, 3400 SEWAGE, Etc)
- Split if there is a difference in frequency of change: Alarms and (high frequency) sensors
- For 'large' topics (number of fields > ~30) see if splitting is possible.
- If there is a large number of fields that are similar in function, having larger topics is acceptable.